In [0]:
dbutils.widgets.text("TableName", "")
dbutils.widgets.text("StageId", "")
dbutils.widgets.text("ProcessInstanceId", "")
dbutils.widgets.text("ProcessQueueId", "")

TableName = dbutils.widgets.get("TableName")
StageId = (dbutils.widgets.get("StageId"))
ProcessInstanceId = (dbutils.widgets.get("ProcessInstanceId"))
ProcessQueueId = (dbutils.widgets.get("ProcessQueueId"))


In [0]:
StageId = int(dbutils.widgets.get("StageId"))
ProcessInstanceId = int(dbutils.widgets.get("ProcessInstanceId"))
ProcessQueueId = int(dbutils.widgets.get("ProcessQueueId"))
TableName = dbutils.widgets.get("TableName")

In [0]:
# Mark current table as InProgress
if StageId == 1:
    spark.sql(f"""
        update control.processqueue
        set ProcessStatus = 'InProgress',
            ProcessStartTime = current_timestamp()
        where StageId = {StageId}
            and ProcessInstanceId = {ProcessInstanceId}
            and ProcessQueueId = {ProcessQueueId}
            and TableName = '{TableName}';
    """)
else:
    print(StageId)
    raise Exception(f"Stage Id is not relavent to ingestion for table: {TableName}")

In [0]:
source_path = f"/Volumes/workspace/source/source-csv"
target_path = f"/Volumes/workspace/raw/raw-volume"

In [0]:
status = False

try:
    print(f"Processing table: {TableName}")

    # Read CSV file from source
    rawdf = (
        spark.read
        .option("header", True)
        .option("inferSchema", True)
        .csv(f"{source_path}/{TableName}.csv")
    )

    table_path = f"{target_path}/{TableName}"

    # Write single CSV
    (
        rawdf.coalesce(1)
        .write
        .format("csv")
        .option("header", "true")
        .mode("overwrite")
        .save(table_path)
    )

    # 🔍 List files INSIDE table folder
    files = dbutils.fs.ls(table_path)

    csv_files = [f.path for f in files if f.name.endswith(".csv")]

    if not csv_files:
        raise Exception("CSV file not generated by Spark")

    # Rename part file to <TableName>.csv
    dbutils.fs.mv(
        csv_files[0],
        f"{table_path}/{TableName}.csv"
    )

    status = True
    print(f"Loaded file successfully: {TableName}.csv")

except Exception as e:
    print(f"Error: {e}")
    status = False

# ---------------------------------
# UPDATE PROCESS QUEUE
# ---------------------------------
if status:
    spark.sql(f"""
        UPDATE control.processqueue
        SET
            ProcessStatus = 'Succeeded',
            ProcessEndTime = current_timestamp(),
            ProcessDuration = CAST(
                (unix_timestamp(current_timestamp()) - unix_timestamp(ProcessStartTime)) / 60
                AS BIGINT
            )
        WHERE
            StageId = {StageId}
            AND ProcessInstanceId = {ProcessInstanceId}
            AND ProcessQueueId = {ProcessQueueId}
            AND TableName = '{TableName}'
    """)
    print(f"{TableName} marked as Successful")

else:
    spark.sql(f"""
        UPDATE control.processqueue
        SET
            ProcessStatus = 'Failed',
            ProcessEndTime = current_timestamp(),
            ProcessDuration = CAST(
                (unix_timestamp(current_timestamp()) - unix_timestamp(ProcessStartTime)) / 60
                AS BIGINT
            )
        WHERE
            StageId = {StageId}
            AND ProcessInstanceId = {ProcessInstanceId}
            AND ProcessQueueId = {ProcessQueueId}
            AND TableName = '{TableName}'
    """)
    print(f"{TableName} marked as Failed")
    raise Exception(f"Hard failure: {TableName} Failure detected")